<a href="https://colab.research.google.com/github/SodiKroehler/MSTHESIS/blob/main/MSTHESIS_tinyllamaOnQbias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
# from langdetect import detect
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from transformers import AutoModel
from transformers import AutoTokenizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
rdf = pd.read_csv("/content/week4_cleaned.csv")
rdf = rdf[rdf['repeatType'] == 'subtitle']

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [9]:
class LlamaMultiClassClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(LlamaMultiClassClassifier, self).__init__()
        self.llama = AutoModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.llama.config.hidden_size, num_classes)  # Single classifier head

    def forward(self, input_ids, attention_mask):
        outputs = self.llama(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state[:, 0, :]  # Take [CLS] token embedding
        logits = self.classifier(hidden_states)
        return logits


In [10]:
class MultiClassDataset(Dataset):
    def __init__(self, queries, labels, tokenizer, max_length=256):
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        text = self.queries[idx]
        inputs = self.tokenizer(text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": self.labels[idx]
        }



In [11]:

label_encoder = preprocessing.LabelEncoder()
rdf["class_id"] = label_encoder.fit_transform(rdf["bias_rating"])

labels = torch.tensor(rdf["class_id"].values, dtype=torch.long)

label_mapping = {idx: label for idx, label in enumerate(label_encoder.classes_)}

dataset = MultiClassDataset(rdf["raw"].tolist(), labels, tokenizer)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [12]:

torch.cuda.empty_cache()


In [ ]:
num_classes = len(label_encoder.classes_)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LlamaMultiClassClassifier("TinyLlama/TinyLlama-1.1B-Chat-v1.0", num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training Loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

In [ ]:
import torch
import os

# Create save directory
save_dir = "tinyllama_classification_model"
os.makedirs(save_dir, exist_ok=True)

# Save model
torch.save(model.state_dict(), os.path.join(save_dir, "model.pth"))

# Save tokenizer
tokenizer.save_pretrained(save_dir)

# Save label encoder (if needed)
import pickle
with open(os.path.join(save_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

print("Model saved successfully!")




In [ ]:
def evaluate_model(model, dataloader, label_encoder):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"

    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].cpu().numpy()  # Ground truth labels

            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=1).cpu().numpy()  # Convert to predicted class index

            true_labels.extend(labels)
            pred_labels.extend(predictions)

    # Convert class IDs back to labels
    true_labels = label_encoder.inverse_transform(true_labels)
    pred_labels = label_encoder.inverse_transform(pred_labels)

    # Print classification report
    print("Classification Report:")
    print(classification_report(true_labels, pred_labels))

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.4f}")

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)

    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()
